In [1]:
!ln -s /Users/adel.schmucklermann/Desktop/FoPra/script/library.py library.py

ln: library.py: File exists


In [2]:
import pytorch_lightning as pl
import h5py
import os
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import library as lib
import pandas as pd

In [3]:
pl.seed_everything(42, workers=True) 

Global seed set to 42


42

# Data Pre-Processing

In [4]:
#make numpy file with (seq header, embedding)
esm2 = h5py.File("/Users/adel.schmucklermann/Desktop/FoPra/KLK/KLK_esm2.h5", 'r')
list(esm2.keys())
# List all groups
keys = list(esm2.keys())
#print(keys) 
# Get the data
embedding = []
headers = []
for key in keys:
    #data
    emb = esm2[key][:]
    embedding.append(emb)
    #headers
    key = key.replace("isoform=", "isoform_").replace("=", "").replace(":", "_")
    headers.append(key)

headers = np.array(headers)
embedding = np.array(embedding)

#split train test
headers_first, headers_test, embedding_first, embedding_test = train_test_split(headers, embedding, test_size=0.1, random_state=42)
#split train validation
headers_train, headers_validate, embedding_train, embedding_validate = train_test_split(headers_first, embedding_first, test_size =0.2, random_state=42)
print("total:", headers.shape)
print("training+validation:", headers_first.shape)
print("test:", headers_test.shape)
print("training:", headers_train.shape)
print("validation:", headers_validate.shape)
print(len(headers_train), len(embedding_train))
print(len(headers_validate), len(embedding_validate))

#training_df = pd.DataFrame({'header': headers_train, 'embedding' : embedding_train})
#validate_df = pd.DataFrame({'header': headers_validate, 'embedding' : embedding_validate})

total: (437,)
training+validation: (393,)
test: (44,)
training: (314,)
validation: (79,)
314 314
79 79


# Dataloaders

In [5]:
train_dataset = lib.makedataset(headers_train, embedding_train)
val_dataset = lib.makedataset(headers_validate, embedding_validate)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
    

# Training VAE

In [21]:
#if prott5 used: dtype=float16 -> error with .linear()

#encoder_layers, latent_dim, decoder_layers
dim = 2560
a = int(dim * 1/16)
b = int(dim * 1/8)
c = int(dim * 1/2)
encoder_layers = [dim, c, b, a]
latent_dim = a
decoder_layers = [a, b, c, dim]

# Initialize the VAE model
vae = lib.VariationalAutoencoder(encoder_layers, latent_dim, decoder_layers)

trainer = pl.Trainer(accelerator="auto", max_epochs=100, check_val_every_n_epoch=1)
#trainer = pl.Trainer(accelerator="gpu", devices = 1, max_epochs=500, check_val_every_n_epoch=1)

# Train the VAE model
trainer.fit(vae, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 3.7 M 
1 | latent  | Sequential | 26.1 K
2 | fc21    | Linear     | 25.8 K
3 | fc22    | Linear     | 25.8 K
4 | fc3     | Sequential | 26.1 K
5 | fc4     | Sequential | 26.1 K
6 | relu    | ReLU       | 0     
7 | decoder | Sequential | 3.8 M 
---------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.491    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


# Validation

In [22]:
trainer.validate(vae, val_dataloader)

Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      validation_loss      │    0.8736222982406616     │
└───────────────────────────┴───────────────────────────┘

[{'validation_loss': 0.8736222982406616}]

# Encode

In [35]:
# Problem with Encode: output is a 2D array
# encoder() or encode()?

#encode = vae.encoder(torch.Tensor(embedding))
mu, logvar = vae.encode(torch.Tensor(embedding))
encode = vae.reparameterize(mu, logvar)
decode = vae.decode(encode)
print(encode)

#encode = encode #[1]


tensor([[-1.5071, -1.5163, -2.8797,  ..., -0.3115, -0.9170, -0.7155],
        [-0.1873,  1.5170, -1.9221,  ..., -1.1345, -2.4209, -0.5521],
        [-1.0468, -0.5463, -0.2903,  ..., -1.0717, -1.7528, -0.1633],
        ...,
        [-1.8191,  1.8874,  0.5434,  ...,  0.8973,  0.2694,  0.8797],
        [-0.1118, -0.3761,  0.3246,  ..., -1.6655, -0.6275, -0.9137],
        [-1.7575,  0.0426, -0.6796,  ..., -0.6944,  0.4838, -1.0702]],
       grad_fn=<AddBackward0>)


# Neighbor_joining and create tree

In [37]:
# Neighbor_joining and create tree

out_newick = "/Users/adel.schmucklermann/Desktop/FoPra/KLK/KLK_prott5_test.newick"


#embeddings_pred = []
#for i in encode:
    #print(len(i))
#    print("i: ",i[0][0])
#    embeddings_pred.append(i[0][0])
#print(embeddings_pred)
    
embedding_out = encode.detach()#torch.stack(encode).detach() 
print(embedding_out)

dist = lib.distance_metric(embedding_out)
distmat = dist.get_metric(embedding_out)
nj = lib.neighbor_joining(distmat,headers)
newick = nj.get_newick(distmat,headers)


with open(out_newick, 'w') as w:
    w.write(newick)
    w.close()

tensor([[-1.5071, -1.5163, -2.8797,  ..., -0.3115, -0.9170, -0.7155],
        [-0.1873,  1.5170, -1.9221,  ..., -1.1345, -2.4209, -0.5521],
        [-1.0468, -0.5463, -0.2903,  ..., -1.0717, -1.7528, -0.1633],
        ...,
        [-1.8191,  1.8874,  0.5434,  ...,  0.8973,  0.2694,  0.8797],
        [-0.1118, -0.3761,  0.3246,  ..., -1.6655, -0.6275, -0.9137],
        [-1.7575,  0.0426, -0.6796,  ..., -0.6944,  0.4838, -1.0702]])


# Plot Loss